In [75]:
import pandas as pd
import json
import requests
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from prophet import Prophet
import asyncio
from dash import Dash, html, dash_table, dcc
import datetime
import time
import os


In [76]:
nasdaq = pd.read_html('Quotes For Nasdaq-100 Index _ Nasdaq.html')[0]

In [77]:
# Função para prever o preço de uma ação


# Com dados de teste 80/20
# 100% reais, tentar prever 20% desses dados já obtidos, para avaliar a precisão do modelo
def forecast_prophet(ds, y):
    data = {'ds': ds, 'y': y}
    df = pd.DataFrame(data)

    # Dividindo os dados em 80/20
    split_index = int(len(df) * 0.8)
    train_data = df[:split_index]
    test_data = df[split_index:]

    # Instanciando o modelo Prophet
    model = Prophet()

    # Ajustando o modelo aos dados de treinamento
    model.fit(train_data)

    # Criando um dataframe com datas futuras para prever (20%)
    future = model.make_future_dataframe(periods=int(len(df) * 0.2))

    # Fazendo previsões
    forecast = model.predict(future)


    return forecast


# Sem dados de teste 80/20
# 80% reais e 20% futuros
def forecast_prophet(ds, y):
    data = {'ds': ds, 'y': y}
    df = pd.DataFrame(data)

    # Instanciando o modelo Prophet
    model = Prophet()
    # Ajustando o modelo aos dados de treinamento
    model.fit(df)

    # Criando um dataframe com datas futuras para prever (20%)
    future = model.make_future_dataframe(periods=int(round(len(data['ds']) *0.2)), freq='D')
    forecast = model.predict(future)

    return forecast


In [78]:
def get_stock_data(ticker, period=None):
    if period:
        stock_data =  yf.download(ticker, period="10y")
    else:
        stock_data =  yf.download(ticker)
    return stock_data

def save_forecast_data(symbol, forecast):
    forecast.to_json(f"prophet-predictions/{symbol}-forecast.json", orient='records')

In [79]:
def main(tickers):
    for ticker in tickers:
        try:
            stock = get_stock_data(ticker)
            # Supondo que 'stock' seja seu DataFrame e contenha uma coluna 'Close'
            # Primeiro, vamos calcular os retornos diários
            stock['Return'] = stock['Close'].pct_change()

            # Em seguida, calculamos o retorno acumulado
            stock['Acc_Return'] = (1 + stock['Return']).cumprod() - 1
            forecast = forecast_prophet(stock.index, stock['Acc_Return'])
            save_forecast_data(ticker, forecast)
            print(f"Saved forecast for {ticker}")

        except Exception as e:
            print(f"Error: {e}")

In [80]:
main(nasdaq.Symbol)

[*********************100%%**********************]  1 of 1 completed
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/_xxt2h3z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/axx9g0xy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=57871', 'data', 'file=/tmp/tmp4vhzzsbg/_xxt2h3z.json', 'init=/tmp/tmp4vhzzsbg/axx9g0xy.json', 'output', 'file=/tmp/tmp4vhzzsbg/prophet_model2wqcroc2/prophet_model-20240415141038.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:10:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:10:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
[*********************100%%**********************]  1 of 1 completed
INFO

Saved forecast for AAPL


DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/boyp9a3m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/czmt1zjp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12524', 'data', 'file=/tmp/tmp4vhzzsbg/boyp9a3m.json', 'init=/tmp/tmp4vhzzsbg/czmt1zjp.json', 'output', 'file=/tmp/tmp4vhzzsbg/prophet_modelcfogcd9p/prophet_model-20240415141051.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:10:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:10:52 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Saved forecast for ABNB


[*********************100%%**********************]  1 of 1 completed
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/tghfijol.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/uc1v1ic5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=76223', 'data', 'file=/tmp/tmp4vhzzsbg/tghfijol.json', 'init=/tmp/tmp4vhzzsbg/uc1v1ic5.json', 'output', 'file=/tmp/tmp4vhzzsbg/prophet_model32qj8sep/prophet_model-20240415141054.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:10:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:10:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


...

Saved forecast for BKNG


[*********************100%%**********************]  1 of 1 completed
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/6a7qbgnn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/_fhua3zg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=47659', 'data', 'file=/tmp/tmp4vhzzsbg/6a7qbgnn.json', 'init=/tmp/tmp4vhzzsbg/_fhua3zg.json', 'output', 'file=/tmp/tmp4vhzzsbg/prophet_modelkwa3zvo_/prophet_model-20240415141356.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:13:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:14:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Saved forecast for BKR


[*********************100%%**********************]  1 of 1 completed
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/k7b35teq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vhzzsbg/vip_v8sx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38928', 'data', 'file=/tmp/tmp4vhzzsbg/k7b35teq.json', 'init=/tmp/tmp4vhzzsbg/vip_v8sx.json', 'output', 'file=/tmp/tmp4vhzzsbg/prophet_model5r2b0v9i/prophet_model-20240415141405.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:14:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


KeyboardInterrupt: 

In [ ]:
#plota os gráficos das previsões junto aos dados reais

def plot_predictions(ticker):


    #pega os dados
    stock = get_stock_data(ticker)


    #prepara o retorno acumulado
    stock['Return'] = stock['Close'].pct_change()
    stock['Acc_Return'] = (1 + stock['Return']).cumprod() - 1

    # preencher nan
    stock.fillna(0, inplace=True)

    #cria forecast
    forecast = forecast_prophet(stock.index, stock['Acc_Return'])

    #Plota gráfico
    plt.figure(figsize=(10, 6))
    plt.plot(pd.to_datetime(stock.index), stock['Acc_Return'], label='Actual', color='black')
    plt.plot(pd.to_datetime(forecast['ds'], unit='ms'), forecast['yhat'], label='Predicted', color='blue')
    plt.fill_between(pd.to_datetime(forecast['ds'], unit='ms'), forecast['yhat_lower'], forecast['yhat_upper'], color='lightblue', alpha=0.3)
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Predictions with Upper and Lower Bounds')
    plt.legend()
    plt.grid(True)
    plt.show()







In [ ]:
plot_predictions('AAPL')

In [ ]:
stock = get_stock_data("MSFT")

In [ ]:

stock['Return'] = stock['Close'].pct_change()
stock['Acc_Return'] = (1 + stock['Return']).cumprod() - 1
stock.fillna(0, inplace=True)

In [ ]:
stock

In [ ]:
stock_forecast = forecast_prophet(stock.index, stock['Acc_Return'])